# House prices 

* trying to learn with help of Pedro Marcelinos notebook link: https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python#'SalePrice',-her-buddies-and-her-interests
Copied a lot by hand as a way to learn. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm 
from sklearn.preprocessing import StandardScaler
from scipy import stats 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
# todo train has the prices already filled in, using the train dataset we are finding the house prices of the test dataset. write smthing like this 
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.columns

### Introduction brainstorm

Try and get overview of the data

What do i value in a house? 

Choose some columns that I want to focus on. 

Focus on:
* SalePrice (the main focus) 
* OverallQual (not sure how it is computed, maybe some sort of average of the other columns conditions)
* YearBuilt 
* TotalBsmtSF
* GrLivArea
* location??

In [ ]:
df_train.describe()

In [ ]:
df_train['SalePrice'].describe()

Look at how sale price relates to some other intresting columns

In [ ]:
#histogram 
sns.distplot(df_train['SalePrice'])

Sales price relative to numerical variables 

In [ ]:
# skewness - todo
# and
# kurtosis - todo 
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

### Relation between SalePrice and some other columns

In [ ]:
#scatter plot grlivarea/saleprice
# todo describe what you see from this graph pls 
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

By looking at the graph I can see that in most cases a bigger GrLivArea (ground living area???) means that is has a higher SalePrice.
It kind of follows a l**inear relationship**

In [ ]:
#scatter plot totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000))

This graph looks to follow more of a exponential shape. 
There are some outlier values, so it could be that the TotalBsmtSF is big but the house has some other flaws that is bringing down the total sales price. 

Looking at categorical features 

In [ ]:
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000)

can see that for the most part the higher the qualiity the more the house costs. 

In [ ]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16,8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000)
plt.xticks(rotation=90)

Can't see a huge differens on the price depending on what year it was built, but I would say that the newer houses tend to cost more. 

small summary - seems like the house prices behave as you mostly would expect. 

In [ ]:
# correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

Heatmaps gives a quick overview of the column relations. 
can see some other interesting columns that have close relation to the sales price that are worth exploring closer...

In [ ]:
# saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

From this we can see more clearly which columns that has a close relation to SalePrice. 
GarageCars and GarageArea impact the SalePrice, they are kind of the same since amout of cars that fit into the garage is depends on how big the garage area is. 
Whether the house has a full bath seem to matter. 
Choosing to discard 'duplicates' such as _______

In [ ]:
# scatterplots between "SalePrice" and correlated variables
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show()

Observation: TotalBsmtSF and GrLivArea, can see a linear line which makes sense because usually basements are the same area as ground living area and not often bigger. Which the graph shows. 
In this scatter plot of SalePrice and YearBuilt i think it shows more clearly that when the house is built more recently the price tend to be more expensive. 

### Missing data 
* How prevalent(accepted) is the missing data? 
* Is missing data random or does it have a pattern? 

In [ ]:
# missing data 
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

When should be consider a column irrelevant? In this case we choose when more than 15 % of the data is missing, that column will be deleted. 
By looking at which columns that are missing the most data, we can see that these columns are not what you typically value when looking for a house. 
Note that the Garage___ columns has the same missing percentage. 
Deciding to delete all variables with missing data except 'Electrical' where we only will delte the observation with missing data. 

In [ ]:
#missing data
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() # doublecheck to make sure no missing data remains 

### Outliers 
Are important and can provide us with good information.
Going to use a quick approch with standard deviation of 'SalePrice' and a set of scatter plots. 

##### Univariate analysis
Looking at one variable at a time. 
Will standardize the data by converting data values to have mean of 0 and a standard deviation of 1. 

In [ ]:
#standardizing data 
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis])
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\n outer range (high) of the distribution:')
print(high_range)

* Low range values are similar and not too far from 0.
* High range values arre far from 0 and the 7 something values are really out of range.

For now, we'll not consider any of these values as an outlier but we should be careful with those two 7 something values. 

##### Bivariate analysis
Going to take a look at previous scatterplots with a new perspective. 

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000))

Revealed:
* Found two values (bigger 'GrLivArea') that are not following the rest. It could be that they are located in a house on the countryside. Regardless they do not follow the typical case so we call them outliers and will delete them. 

* The two highest SalePrice points are a bit out from the rest but still seem to follow the trend so we will keep them. 

In [ ]:
# deleting points 
df_train.sort_values(by= 'GrLivArea', ascending = False)[:2]
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
# bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000))

You might be tempted to delete the TotalBsmtSf > 3000 points but decided to leave them in. 

### How is SalePrice calculated? 

testing four assumptions:

Normality: The data should follow a normal distribution. 

Homoscendasticity: refers to the 'assumption that dependent variable(s) exhibit equal levels of variance across the range of predictor variable(s)'. Homoscedasticity is desirable because we want the error term to be the same across all values of the independet variables. 

Linearity: The most common way is to examine scatter plots and search for linear patterns. If patterns are not linear, it would be worthwhile to explore data transformations. However, we'll not get into this because most of the scatter plots we've seen appear to have linear relationships. 

Abscence of correlated errors: They happen when one error is correlated to another. For instance, if one positive error makes a negative error systematiaclly, it means that there's a relationship between these variables. This occurs often in time series, where some patterns are time related. When getting this error, try to add a variable that can explain the effect you are getting. That's the most common solution for correlated errors.

In [ ]:
# Exploring normality
# histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

'SalePrice' is not normal. It shows 'peakedness', positive skewness and does not follow the diagonal line. 

But we can use data transformation to solve the problem. There is a 'trick' where in the case of positive skewness, log transformations usually works well. 

In [ ]:
# applying log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
# the transformed histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

Great! Now lets see 'GrLivArea'

In [ ]:
# 'GrLivArea' histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
# It is skewed so... data transformation
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [ ]:
# the transformed histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm)
fig =plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

On to the next couple! 

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot = plt)

* Seeing skewness.
* Some zero values indicating houses without basements.
* Zero values preventing the log transformation to take place. 

There is a way to work through this problem. We will create a variable taht can get the effect of having or not having a basement (binary variable). Then we will do log transformation to all the non-zero observations, ignoring those with value zero. This way we can transform data, without losing the effect of having a basement or not. 

In [ ]:
# create column for new variable (one is enough because it's a binary categorical feature)
# if area > 0 it gets 1, if area is == 0 it gets 0
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0
df_train.loc[df_train['TotalBsmtSF'] > 0, 'HasBsmt'] = 1

In [ ]:
# transform data
df_train.loc[df_train['HasBsmt'] == 1, 'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], plot=plt)

#### Homoscedasticity

The best approch to test homoscedasticity for two metric variables is graphically. Departures from an equal dispersion are shown by such shapes as cones (small dispersion at one side of the graph, larger dispersion at the opposite side) or diamonds ( a large number of points at the center of the distribution).

In [ ]:
# starting by SalePrice and GrLivArea
# scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice'])

The scatter plot before log transformations, had a conic shape. Now it does'nt have a conic shape, thats the power of normality. 

In [ ]:
# SalePrice with TotalBsmtSF
# scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF'] > 0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF'] > 0]['SalePrice'])

With this we can say in general that 'SalePrice' exhibit equal levels of variance across the range of 'TotalBsmtSF'

In [ ]:
# covert categorical variable into dummy
# dummy variables are categorical data turned into 1 or 0 for simplicity. 
df_train = pd.get_dummies(df_train)